# Project Description

This project is based in the section about Meta programming of the paper [1].

Before you start, we recommend you to read that section, up to part 3.3.2 (included),
and to do the exercises at:
* [../meta-programming-exercises/meta-programming-exercises.ipynb](../meta-programming-exercises/meta-programming-exercises.ipynb)

We start with some preliminaries, and then describe the two parts of this project:
* the first is about **computing weighted diverse answer sets**, and
* the second is about **building a SAT-based ASP solver**.

[1] Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2020). How to build your own ASP-based system?! CoRR, abs/2008.06692.

## Preliminaries

Let us consider the Latin Square problem:
given a quadratic board of size `s`, 
the goal is to fill each cell of the
board with some number from `1` to `s` such that 
no number occurs twice in the same row or column.

This problem can be represented by the following logic program:

In [1]:
%%file latin.lp
number(1..s).

1 { latin(X,Y,N) : number(N) } 1 :- number(X), number(Y).
1 { latin(X,Y,N) : number(Y) } 1 :- number(X), number(N).
1 { latin(X,Y,N) : number(X) } 1 :- number(Y), number(N).

#show latin/3.

Overwriting latin.lp


The paper [1] presents the following meta encoding `meta.lp`.

In [2]:
%%file meta.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

Overwriting meta.lp


The file can be used to compute answer sets of a logic program `P`
given a reified version of the program `P` generated by `clingo`.
For example, the answer sets generated by this command:

In [3]:
!clingo latin.lp -c s=4 0 --quiet

clingo version 5.7.1
Reading from latin.lp
Solving...
SATISFIABLE

Models       : 576
Calls        : 1
Time         : 0.003s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.002s


are the same as those generated by these ones:

In [4]:
!clingo latin.lp -c s=4 --output=reify > reify.lp
!clingo reify.lp meta.lp 0 --quiet

clingo version 5.7.1
Reading from reify.lp ...
Solving...
SATISFIABLE

Models       : 576
Calls        : 1
Time         : 0.012s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.011s


You can check this equivalence using the following Python function.
It takes as input a string and some files that, together, specify a logic program.
The function returns a sorted version of the answer sets of the program.

In [5]:
import clingo

def run(program="", files=[]):
    ctl = clingo.Control(["0"])
    ctl.add("base", [], program)
    for f in files:
        ctl.load(f)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        return sorted(sorted(list(m.symbols(shown=True))) for m in handle)

Note that the answer sets returned consist only of atoms and terms that are shown.

We can check that the previous programs have the same answer sets as follows:

In [6]:
!clingo latin.lp -c s=4 --output=reify > reify.lp
run("", ['reify.lp', 'meta.lp']) == run("#const s=4.", ['latin.lp'])

True

The comparison returns `True` because the answer sets of both programs are the same.

**Hint**: When you debug a meta-encoding
* try to come up with the minimal example that triggers the problem, and
* work on this simple example before trying some bigger program.

## Part 1: Computing weighted diverse optimal models 

The paper [1] presents the following meta encoding for computing several diverse answer sets:

In [ ]:
%%file many.lp
model(1..m).

conjunction(B,M) :- model(M), literal_tuple(B),
        hold(L,M) : literal_tuple(B, L), L > 0;
    not hold(L,M) : literal_tuple(B,-L), L > 0.

body(normal(B),M) :- rule(_,normal(B)), conjunction(B,M).
body(sum(B,G),M)  :- model(M), rule(_,sum(B,G)),
    #sum { W,L :     hold(L,M), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L,M), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A,M) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B,M).
{ hold(A,M) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B,M).

show(T,M) :- output(T,B), conjunction(B,M).
#show.
#show (T,M) : show(T,M).

#const k=1.

:- model(M), model(N), M<N, option = 1,
    #sum{ 1,T: show(T,M), not show(T,N) ;
          1,T: not show(T,M), show(T,N) } < k.

#maximize{ 1,T,M,N: show(T,M), not show(T,N), model(N), option = 2 }.

The task of this part is to write a meta encoding `weighted-many.lp`, 
extending `many.lp`,
for computing weighted diverse answer sets.

In this case the input logic program is accompanied by 
a set of facts about predicate `weight/2`, assigning weights to atoms:
```
weight(A,W). % The weight of atom A is W
```
Then the distance between two answer sets is the sum of the weights of the atoms 
that are interpreted differently in the two answer sets.

The notions of `k`-diverse and most-diverse answer sets are defined as in [1], 
using the new distance measure instead of the Hamming distance.

As a result, the notions of [1] correspond to the special case of this setting
where all atoms are given a weight of `1`.


In the directory `diverse-instances` you can find 10 instances. 

They consist of the `cells.lp` encoding of [1] (using a `3x3` grid and marking `7` cells) 
together with different sets of facts about predicate `weight/2`.

Your meta encoding should work with all those instances. 

More specifically, for each instance `ins.lp`, the following command should return the sets of `3` weighted most-diverse answer sets (they appear commented at the end of every instance file):

In [ ]:
!clingo ins.lp --output=reify > reify.lp
!clingo reify.lp weighted-many.lp -c m=3 -c option=2 --opt-mode=optN 0 --quiet=1

You can use the `test.py` script to check your encoding over all instances.

The script does not give you much feedback, 
so we recommend you to use it only once you have seen that your encoding works on some instances
(for example, trying the commands above).

In [ ]:
! python test.py 60 many

## Part 2. Building a SAT-based ASP solver

An ASP solver is a computer program that finds answer sets of logic programs.

A SAT solver is a computer program that finds models of propositional formulas.

A SAT-based ASP solver is a computer program that finds answer sets of some logic program `P` by:
* translating `P` into a propositional formula `F`
such that there is a correspondence between the models of `F` 
and the answer sets of `P`, and
* running a SAT solver on the formula `F`. 

Given the correspondence between `P` and `F`, 
the models of `F` found by the SAT solver can be mapped to answer sets of `P`:
* the SAT solver finds answer sets of `P`!

Since there are many efficient SAT solvers, like [kissat](https://fmv.jku.at/kissat), this may be an effective approach to ASP solving.

The input to SAT solvers is usually in the [DIMACS](http://www.satcompetition.org/2009/format-benchmarks2009.html) format. Here is an example:

In [7]:
%%file example1.cnf
c
c some comments
c 
p cnf 3 4
1 -2 0
2 -1 0
3 -1 -2 0
3  1  2 0

Overwriting example1.cnf


The file can start with some comments, that begin with the character `c`.

After the comments, there is a line of the form `p cnf <n> <c>` where:
* `<n>` is the number of atoms occurring in the file, and
* `<c>` is the number of clauses (see below) occurring in the file.

Every remaining line is a clause, 
that is a sequence of non-zero numbers (atoms) between `-<n>` and `<n>` terminated by zero.

The example above has 4 clauses, that in propositional logic (considering `1`, `2` and `3` as atoms) could be written as:
* $1 \vee \neg 2$
* $2 \vee \neg 1$
* $3 \vee \neg 1 \vee \neg 2$
* $3 \vee 1 \vee 2$

The models of `example1.cnf` are the models of those 4 clauses: `{1,2,3}` and `{3}`.

They can be computed by clingo using option `--mode=clasp`. Yes, clingo is also a SAT solver!

In [10]:
! clingo --mode=clasp example1.cnf 0 -V0

v -1 -2 3 0
v 1 2 3 0
s SATISFIABLE


The clauses can also be represented as negations of conjunctions, that resemble integrity constraints:
* $\neg (\neg 1 \wedge 2)$
* $\neg (\neg 2 \wedge 1)$
* $\neg (\neg 3 \wedge 1 \wedge 2)$
* $\neg (\neg 3 \wedge \neg 1 \wedge \neg 2)$

This leads us to a simple translation from DIMACS to ASP, 
using choice rules to generate the atoms, 
and one integrity constraint per clause:

In [8]:
%%file example1.cnf.lp
{ hold(1..3) }.
:- not hold(1),     hold(2).              % 1 -2 0
:- not hold(2),     hold(1).              % 2 -1 0
:- not hold(3),     hold(1),     hold(2). % 3 -1 -2 0
:- not hold(3), not hold(1), not hold(2). % 3  1  2 0

Overwriting example1.cnf.lp


The answer sets of this logic program correspond to the models of the previous formula.

In [9]:
! clingo example1.cnf.lp 0 -V0

hold(3)
hold(1) hold(2) hold(3)
SATISFIABLE


This translation from DIMACS (`example1.cnf`) to ASP (`example1.cnf.lp`) 
shows us also a translation from a fragment of ASP to DIMACS.

If we have a logic program (like `example1.cnf.lp`) that consists only of:
* simple choice rules (without bounds), and
* integrity constraints over atoms or negated atoms

we can translate it to a DIMACS file, like `example.cnf`, simply by:
* adding the appropriate header `p cnf <n> <c>`, and
* translating every integrity constraint to a clause.

We can then compute the answer sets of the original logic program
by computing the models of the DIMACS file using a SAT solver 
(like clingo with option `--mode=clasp`, or [kissat](https://fmv.jku.at/kissat)).

To build a SAT-based ASP solver there is only one missing piece.

We have to translate:
* from logic programs in general
* to logic programs in that restricted form (choice rules plus integrity constraints)

Once this is done, we will have a SAT-based ASP solver, 
that translates from logic programs in general to DIMACS, and runs our SAT solver of choice.

We separate the whole translation in 3 Tasks:
* Task 1 translates a logic program `P1`, with choice rules and weight rules, to a normal logic program `P2` without them.
* Task 2 translates a normal logic program `P2` to a program `P3` in restricted form (choice rules plus integrity constraints).
* Task 3 translates a logic program `P3` in restricted form (choice rules plus integrity constraints) to DIMACS.

Tasks 1 and 2 shoud be implemented using meta-programming in clingo.

Task 3 should be implemented in your language of choice.

### Task 1: translating choice rules and weight rules

The goal of Task 1 is to write a meta encoding `meta-normal.lp` 
whose ground instantiation (generated by clingo) has no choice rules or weight rules.

The new meta encoding `meta-normal.lp` should behave like `meta.lp`, so that
it can be used to compute answer sets of a logic program `P`
given a reified version of the program `P` generated by clingo.

In [ ]:
%%file meta-normal.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
    
%
% Replace this rule:
%
% body(sum(B,G))  :- rule(_,sum(B,G)),
%     #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
%            W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).

%
% Replace this rule:
%
% { hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

Once it is ready, you can run the same test as above:

In [ ]:
!clingo latin.lp -c s=4 --output=reify > reify.lp
run("", ['reify.lp', 'meta-normal.lp']) == run("#const s=4.", ['latin.lp'])

You can check if the ground instantiation of the meta encoding has no choice rules or weight rules with this call:

In [ ]:
!clingo latin.lp         -c s=4 --output=reify > reify.lp
!clingo reify.lp meta-normal.lp --output=reify | grep -E '^weighted_literal_tuple|^rule\(choice'

If the meta encoding is correct, then the output should be empty.

In the directory ``translation-instances`` you can find 10 instances. They consist of the `latin.lp` encoding together with one additional aggregate, whose bounds are different along the instances. Your meta encoding should work with all those instances. More specifically, for each instance `ins.lp`, the following cell should return `True`.

In [ ]:
!clingo ins.lp --output=reify > reify.lp
run("", ['reify.lp', 'meta-normal.lp']) == run("", ['ins.lp'])

You can check all instances at once using the `test.py` script:

In [ ]:
! python test.py 60 normal

#### Translating choice rules

The meta-encoding `meta.lp` contains the rule
```
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).
```
that generates the atoms of a choice rule whenever its body holds.
The first step of this part is to replace that rule by a set of normal rules.

In our course Answer Set Solving in Practice (https://teaching.potassco.org), 
in the Language part, we explain how to do this for ground logic programs.
You can apply the same approach here.

#### Translating weight rules

The meta-encoding `meta.lp` contains the rule
```
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.
```
that determines when the body of a weight rule holds, using a `#sum` aggregate in the body.
The second step of this part is to replace that rule by a set of normal rules.

In our course Answer Set Solving in Practice (https://teaching.potassco.org),
in the Language part, 
we explain how to do this for cardinality rules occurring in ground logic programs.
You can apply the same approach here, except that in this case, 
whenever a literal holds, 
instead of adding `1` to the accumulated values, 
we have to add the corresponding weight `w` of the literal.

The approach requires to order the literals occurring in the weight rules.
That can be done with this rule:
```
weighted_literal_tuple(B,L,W,P) :- weighted_literal_tuple(B,L,W),
                                   P = #count{ LL,WW : weighted_literal_tuple(B,LL,WW), (LL,WW)<=(L,W) }. 
```
The argument `P` of `weighted_literal_tuple` gives the order of the literal `L` with weight `W` in the weight rule `B`. Observe that the rule uses an aggregate, 
but since it only depends on the facts about `weighted_literal_tuple/3`,
clingo can simplify the rule and ground it into facts.  
The rule is not normal, 
but given that its ground instantiation has no choice rules or weight rules, 
you can (and should) use it in your meta encoding.

**Note**: The weights generated by `clingo` in the reified output are always positive integers.
In other words, the third argument of the atoms of the predicate `weighted_literal_tuple/3` is always a positive integer.

### Task 2. Translating to choice rules plus integrity constraints

The task of this part is to write a meta encoding `meta-constraints.lp` 
whose ground instantiation (generated by `clingo`) has only choice rules and integrity constraints.

We restrict the input programs to normal programs **with integrity constraints**.
These programs do not contain disjunctions, choice rules or weight rules. 
Furthermore, the input programs have to be tight, i.e., 
the positive atom dependency graph of the ground instantiation of the programs must be acyclic. 

Note that these are the programs that we obtain from Part 1
if the original program is tight and non-disjunctive, 
as in the case of `latin.lp`.
To see this, please inspect the output of the next cell:

In [ ]:
!clingo latin.lp         -c s=4 --output=reify > reify.lp
!clingo reify.lp meta-normal.lp --output=reify

The new meta encoding `meta-constraints.lp` should behave like `meta.lp`, so that
it can be used to compute answer sets of a tight normal logic program `P` with integrity constraints,
given a reified version of the program `P` generated by `clingo`.
You can run a similar test as before:

In [ ]:
!clingo latin.lp          -c s=4 --output=reify > reify1.lp
!clingo reify1.lp meta-normal.lp --output=reify > reify2.lp
run("", ['reify2.lp', 'meta-constraints.lp']) == run("#const s=4.", ['latin.lp'])

In the course Answer Set Solving in Practice (https://teaching.potassco.org), 
in the Solving part, 
we show how to represent a tight logic program by a set of nogoods
such that the answer sets of the logic program correspond 
to the solutions to the nogoods.
For tight logic programs, those nogoods are called the completion nogoods.

The answer sets of the meta encoding `meta-constraints.lp` should correspond to 
the solutions to the completion nogoods.
The choice rules are used to generate all possible assignments, while
the constraints represent the completion nogoods.
For example, the nogood `{Ta, Fb, Fc}` can be represented by the constraint `:- a, not b, not c.`.
Then, given the previous result from the course, 
the answer sets of the meta encoding will correspond to the answer sets of the original logic program, 
and this part will be completed.

We already provide you with the choice rules and the show statements that you need:

In [ ]:
%%file meta-constraints.lp

{conjunction(B)} :- literal_tuple(B).
{     hold(|L|)} :- literal_tuple(B,L).
{     hold( A )} :- atom_tuple(H,A).

% Add the constraints here...
    
#show.
#show T : output(T,B), conjunction(B).

Your task is to add the constraints representing the nogoods.
You only need two constraints for the atom-oriented nogoods, and
three constraints for the body-oriented nogoods.

Your meta encoding should work with the 10 instances of the directory ``translation-instances``. More specifically, for each instance `ins.lp`, the following cell should return `True`.

In [ ]:
!clingo ins.lp                   --output=reify > reify1.lp
!clingo reify1.lp meta-normal.lp --output=reify > reify2.lp
run("", ['reify2.lp', 'meta-constraints.lp']) == run("", [''ins.lp'])

You can check all instances at once using the `test.py` script:

In [ ]:
! python test.py 60 constraints

Note that this option of `test.py` uses the encoding `meta-constraints.lp` *and* the encoding `meta-normal.lp`.
Hence, the test will only be successful if both encodings are correct.

### Task 3: Translating to DIMACS

The goal of this task is to implement a translator `asp2dimacs`:
* from logic programs in aspif format that consist only of:
  - facts
  - choice rules with one element in the head and an empty body
  - integrity constraints whose atoms occur in some choice rule
* to DIMACS files

such that the answer sets of the aspif programs are the same as the models of the translated DIMACS files.

You can implement this translator with your language of choice.

As an example, this command:
```
clingo example1.cnf.lp --mode=gringo | asp2dimacs
```
should print a file like `example1.cnf`.

To solve the latin squares problem, you could run these commands:

In [ ]:
!clingo latin.lp          -c s=4 --output=reify > reify1.lp
!clingo reify1.lp meta-normal.lp --output=reify > reify2.lp
!clingo reify2.lp meta-constraints.lp --mode=gringo | asp2dimacs | clingo --mode=clasp

#### An issue with conditional literals

There may be a problem with the last line if the program 
meta-constraints.lp has conditional literals in the body, 
as it will probably have. 

Consider this program:

In [1]:
%%file conditional.lp
{ a(1..3) }.
:- a(1), not a(X) : X=2..3.
{ b(1..3) }.
:- not b(1), b(X) : X=2..3.
#show.

Writing conditional.lp


In [2]:
! clingo conditional.lp --mode=gringo

asp 1 0 0
1 1 1 1 0 0
1 1 1 2 0 0
1 1 1 3 0 0
1 0 1 4 0 2 2 3
1 0 0 0 2 4 -1
1 1 1 5 0 0
1 1 1 6 0 0
1 1 1 7 0 0
1 0 1 8 0 2 -6 -7
1 0 0 0 2 8 5
0


This corresponds to the ground program:
```
{ 1 }.
{ 2 }.
{ 3 }.
4 :- 2, 3.
:- 4, not 1.
{ 5 }.
{ 6 }.
{ 7 }.
8 :- not 6, not 7.
:- 8, 5.
```
Observe that instead of printing directly the constraints:
```
:- 2, 3, not 1.
:- not 6, not 7, 5.
```
clingo generates for each of them two rules: 
* a normal rule `4 :- 2, 3.` and an integrity constraint `:- 4, not 1.` for the first one, and
* a normal rule `8 :- not 6, not 7.` and an integrity constraint `:- 8, 5.` for the second one.

Then, if `asp2dimacs` only handles facts, choice rules and integrity constraints, 
it will not be able to process
the result of: 
* `clingo conditional.lp --mode=gringo`.

Since your meta-encoding `meta-constraints.lp` may also have 
integrity constraints with conditional literals, like `:- a(1), not a(X) : X=2..3.`, 
it may also not be able to process the result of:
* `clingo reify2.lp meta-constraints.lp --mode=gringo`.

To avoid this,
your script `asp2dimacs` should be able to handle this *specific* case
where an atom is defined by a normal rule, and then it occurs in an integrity constraint
in the next line.
For this project, you can assume that this defined atom does not occur anywhere else in the program.


In our example, it should translate together:
* `4 :- 2, 3.` and `:- 4, not 1.` into `-2 -3 1 0`, and
* `8 :- not 6, not 7.` and `:- 8, 5.` into `6 7 -5 0`.

We **do not care** about the truth value of the atoms `4` and `8`,
but your translation should result in the same number of solutions as the original program,
hence you cannot let them free.

A simple approach is to make them true with the clauses `4 0` and `8 0`.
In this case, the call
```
clingo conditional.lp --mode=gringo | asp2dimacs 
```
should print this:
```
p cnf 8 4
4 0
-2 -3 1 0
8 0
6 7 -5 0
```

Regards testing, for this project 
it is enough if you test that your implementation computes the right number of solutions.

For example, for every instance `instance0x.lp` in `translation-instances`, 
you can check if the number of answers (which is already written in the instance file)
is the same as the number of solutions computed by the whole call:

In [ ]:
!clingo instance0x.lp --output=reify > reify1.lp
!clingo reify1.lp meta-normal.lp --output=reify > reify2.lp
!clingo reify2.lp meta-constraints.lp --mode=gringo | asp2dimacs | clingo --mode=clasp 0 | grep Models

### Extensions

* The script `asp2dimacs` could be extended to translate the model found by the SAT solver into its symbolic representation, 
  i.e., instead of printing `v -1 -2 3 0` it could print `hold(3)`.
* The meta-encodings `meta-normal.lp` and `meta-constraints.lp` could be combined into a unique meta-encoding.
* The translation in `meta-normal.lp` of weight rules and/or integrity constraints could be made more efficient (see [2]).
* The translation in `meta-constraints.lp` could handle also non-tight logic programs (see [2,3]).

[2] Tomi Janhunen, Ilkka Niemelä: Compact Translations of Non-disjunctive Answer Set Programs to Propositional Clauses. Logic Programming, Knowledge Representation, and Nonmonotonic Reasoning 2011: 111-130

[3] Masood Feyzbakhsh Rankooh, Tomi Janhunen: Efficient Computation of Answer Sets via SAT Modulo Acyclicity and Vertex Elimination. LPNMR 2022: 203-216